In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh
from scipy.sparse import csr_matrix
import time

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
#adjacency_matrix.data = np.ones_like(adjacency_matrix.data, dtype=np.float64)
adjacency_matrix.data = adjacency_matrix.data ** 0.1

In [3]:
n = adjacency_matrix.shape[0]
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
D = scipy.sparse.spdiags(degrees, [0], n, n, format='csr')
L = D - adjacency_matrix
degrees_sqrt = np.sqrt(degrees)
D_inv_sqrt = scipy.sparse.spdiags(1.0 / degrees_sqrt, [0], n, n, format='csr')
L = D_inv_sqrt.dot(L.dot(D_inv_sqrt))

In [4]:
I = scipy.sparse.eye(n, n, dtype=np.float64, format='csr')

In [4]:
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [3]:
# PMI-adjacency
n = adjacency_matrix.shape[0]
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
total_count = degrees.sum()
D_inv = scipy.sparse.spdiags(1.0 / degrees, [0], n, n, format='csr')
adjacency_matrix = D_inv.dot(adjacency_matrix.dot(D_inv))
adjacency_matrix.data = np.maximum(np.log(adjacency_matrix.data * total_count), 0)
adjacency_matrix.eliminate_zeros()
degrees = np.asarray(adjacency_matrix.sum(axis=1), dtype=np.float64).flatten() #Update degrees

In [5]:
#r = np.sqrt(5176.292450)
r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
n = adjacency_matrix.shape[0]

dt = adjacency_matrix.data * r
dt /= r**2 - adjacency_matrix.data ** 2

adjacency_matrix.data = adjacency_matrix.data**2 / (r**2 - adjacency_matrix.data**2)
bethe_diagonal = 1 + np.asarray(adjacency_matrix.sum(axis=1)).flatten()
D = scipy.sparse.spdiags(bethe_diagonal, [0], n, n, format='csr')

adjacency_matrix.data = dt
Hr = D - adjacency_matrix

In [6]:
#rng = np.random.RandomState(0)
dim = 100
tol = np.sqrt(1e-15)*n
#init = rng.rand(n, dim)
#init[:,0] = degrees_sqrt

start = time.time()
#vals, vecs = lobpcg(L, X=init, maxiter=100, largest=False, verbosityLevel=1)
# vals, vecs = eigsh(L-I, dim, which='LM', tol=tol)
vals, vecs = eigsh(Hr, dim, which='SA', tol=tol)
print("time elapsed: %d" % (time.time() - start))

time elapsed: 388


In [7]:
vals

array([-1258.77790519,  -270.77964403,  -250.29773191,  -211.15745687,
        -207.83040302,  -135.72468745,  -126.8875453 ,  -115.86487585,
        -107.55176857,  -101.82689345,   -75.62324121,   -71.30029337,
         -64.48846634,   -59.66877346,   -57.40447959,   -57.24259512,
         -53.93979442,   -48.98313316,   -46.84369598,   -42.91140712,
         -38.35490698,   -37.08276147,   -33.98709954,   -29.21813586,
         -29.00570564,   -28.39938261,   -27.21149534,   -26.64904835,
         -25.03972875,   -22.26910664,   -18.45135919,   -17.17669717,
         -16.46069154,   -15.28867459,   -14.42701873,   -14.20483487,
         -13.57182498,   -11.77164402,   -11.39343534,   -10.43915631,
          -9.78811498,    -9.62494088,    -9.31375842,    -8.59368845,
          -8.499044  ,    -8.39754516,    -8.06637818,    -7.86229649,
          -7.79291582,    -7.63039773,    -7.47575554,    -7.23172879,
          -6.75923199,    -6.41215975,    -6.37555006,    -6.15992427,
      

In [8]:
output_path = "../data/wiki/win=1_weighted_bethe_hessian_pow=0.50_dim=100"
np.save(output_path + ".vecs", vecs)
np.save(output_path + ".vals", vals)
np.save(output_path + ".degrees", degrees)

In [11]:
output_path = "../data/wiki/win=1_bethe_hessian_small_rhoB_est_pow=0.00_dim=100"
vecs2 = np.load(output_path + ".vecs.npy")
vals2 = np.load(output_path + ".vals.npy")

In [14]:
all_vals = np.concatenate([vals, vals2], axis=0)
all_vecs = np.concatenate([vecs, vecs2], axis=1)

In [19]:
top_vals_inds = np.argsort(all_vals)[:100]
output_path = "../data/wiki/win=1_bethe_hessian_combo_rhoB_est_pow=0.00_dim=100"
np.save(output_path + ".vecs", all_vecs[:, top_vals_inds])
np.save(output_path + ".vals", all_vals[top_vals_inds])
np.save(output_path + ".degrees", degrees)